# Corona Trends in Denmark, Sweden and Switzerland

The following tables and figures display Covid-19 data for Denmark, Sweden and Switzerland. Data is taken from the John Hopkins University [github repository](https://github.com/CSSEGISandData/COVID-19). The dynamics for these countries clearly differ...

Update
- 6.5.2020: Italy included for comparison

In [62]:
from IPython.display import HTML

In [63]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [19]:
# jupyter nbconvert --to html --TemplateExporter.exclude_input=True --no-prompt Denmark_Sweden_Switzerland.ipynb
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.signal import savgol_filter

from corona import DataDownloader
from corona import apply_to_column as a2c

In [26]:
population = pd.Series(name='population',data=
    {'Denmark':5806000,
     'Sweden':10230000,
     'Switzerland':8570000,
     'Italy':60360000})

In [64]:
country_df = pd.DataFrame(columns = ['country','population','color','visible'],data=
    [['Denmark',5806000,'black',True],
     ['Sweden',10230000,'blue',True],
     ['Switzerland',8570000,'red',True],
     ['Italy',60360000,'green','legendonly']]).set_index('country')
#country_df

In [54]:
countries = ['Switzerland','Sweden','Denmark']
countries = country_df.index.tolist()

# load data and aggregate over states (elements of countries)
df = (
    DataDownloader().read_jhu()
    .query('country in @countries')
    .groupby(['country','serie','date'])
    .sum()
    .reset_index() 
    .sort_values(by=['country','serie','date']) )

In [55]:
# calculate number per 1 million
df['cumulative1mio'] = df.cumulative / country_df.population.reindex(df.country).values * 1e6
df['daily1mio'] = df.daily / country_df.population.reindex(df.country).values * 1e6

## Statistics

A few overall numbers. Observations
- The ratio of confirmed to deaths cases is double in Sweden and Italy is double than in Switzlerland and Denmark

In [56]:
statdf = (
    df.groupby(['country','serie'])[['cumulative']]
    .max()
    .dropna()
    .astype(int)
    .reset_index()
    .pivot(index='country',columns='serie',values='cumulative')
    .sort_values('deaths',ascending=False)  
    .join(population)
    .assign(confirmed_1mio=lambda df: df.confirmed / df.population * 1e6)
    .assign(deaths_1mio=lambda df: df.deaths / df.population * 1e6)
    .astype(int)
)

statdf['deaths_confirmed_percent'] =  np.round(statdf.deaths/statdf.confirmed *100,1)
statdf.head()

,confirmed,deaths,population,confirmed_1mio,deaths_1mio,deaths_confirmed_percent
country,,,,,,
Italy,213013,29315,60360000,3529,485,13.8
Sweden,23216,2854,10230000,2269,278,12.3
Switzerland,30009,1795,8570000,3501,209,6.0
Denmark,10019,503,5806000,1725,86,5.0


## Daily cases

Trend lines are running averages over 1 week in order to smooth out variations due to reporting breaks on weekends.

**All plots are interactive:** 
- Click on the legend to hide/activate data
- Zoom in 
- ...

### Total per country

In [57]:
def trend(s):
    if len(s) >= 7:
        out = s.replace({0:1}).rolling(7,center=True,win_type='boxcar',min_periods=1).mean()
        return pd.Series(out, index=s.index)
    else:
        return s * np.nan

df['daily_trend'] = a2c(df,'daily',trend)
df['daily1mio_trend'] = a2c(df,'daily1mio',trend)

In [67]:
for serie in df.serie.unique():
    fig = go.Figure()
    for country in df.country.unique():
        sdf = df.query('serie==@serie & country ==@country') 
        color = country_df.color[country]
        visible = country_df.visible[country]
       
        fig.add_bar(x=sdf.date, y=sdf.daily, marker_color = color, 
                    name = country, visible = visible)
        
        fig.add_scatter(x=sdf.date, y=sdf.daily_trend, mode='lines',line={'color':color}, 
                        name = 'weekly average', visible=visible)
    fig.update_layout(
        legend=dict(x=0.05, y=0.9),
        title ='total number of '+serie+' cases')
    
    fig.show()

### Per 1 million inhabitants

Sweden has currently around 6 deaths per day and 1 million inhabitant. In Switzerland and Denmark the same number is around 2.  

In [66]:
for serie in df.serie.unique():
    fig = go.Figure()
    for country in df.country.unique():
        sdf = df.query('serie==@serie & country ==@country') 
        color = country_df.color[country]
        visible = True #country_df.visible[country]
       
        fig.add_bar(x=sdf.date, y=sdf.daily1mio, marker_color = color, 
                    name = country, visible = visible)
        
        fig.add_scatter(x=sdf.date, y=sdf.daily1mio_trend, mode='lines',line={'color':color}, 
                        name = 'weekly average', visible=visible)
    fig.update_layout(
        legend=dict(x=0.05, y=0.9),
        title =serie+' cases per 1 million inhabitants')
    
    fig.show()